In [8]:
import subprocess, os, textwrap

def run(cmd):
    print(f"\n$ {cmd}")
    proc = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    print("=== STDOUT ===")
    print(proc.stdout.strip() or "<none>")
    print("=== STDERR ===")
    print(proc.stderr.strip() or "<none>")
    print(f"returncode: {proc.returncode}")
    return proc

os.chdir("/data/ct-clip-app")

# Ensure identity
run('git config user.name "Mustafa Ak"')
run('git config user.email "mustafaxacikgoz@gmail.com"')

# Show status
run("git status")

# Stage artifacts
run("git add saved_artifacts/features_labels.pt saved_artifacts/sample_filenames.txt saved_artifacts/training_summary.csv")

# Commit if needed
run('git commit -m "Add extracted features, filename manifest, and summary" || echo "Nothing to commit"')

# Push (with token if needed)
# Try normal push first
res = run("git push origin main")
if res.returncode != 0:
    # Retry with embedded token from secret (adjust secret name if different)
    token = os.environ.get("GITHUB_TOKEN") or os.environ.get("hf_space_backup")
    if not token:
        print("No GitHub token found in environment secrets; cannot retry authenticated push.")
    else:
        run(f"git remote set-url origin https://{token}@github.com/Mustafa-Acikgoz/ct-clip-app.git")
        run("git push origin main")



$ git config user.name "Mustafa Ak"
=== STDOUT ===
<none>
=== STDERR ===
<none>
returncode: 0

$ git config user.email "mustafaxacikgoz@gmail.com"
=== STDOUT ===
<none>
=== STDERR ===
<none>
returncode: 0

$ git status
=== STDOUT ===
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   CT-CLIP/CT_CLIP/ct_clip/__pycache__/__init__.cpython-39.pyc
	modified:   CT-CLIP/CT_CLIP/ct_clip/__pycache__/ct_clip.cpython-39.pyc
	modified:   CT-CLIP/CT_CLIP/ct_clip/__pycache__/mlm.cpython-39.pyc
	modified:   CT-CLIP/CT_CLIP/ct_clip/__pycache__/visual_ssl.cpython-39.pyc
	modified:   CT-CLIP/ct-clip.ipynb
	modified:   CT-CLIP/transformer_maskgit/transformer_maskgit/__pycache__/MaskGITTransformer.cpython-39.pyc
	modified:   CT-CLIP/transformer_maskgit/transformer_maskgit/__pycache__/__init__.cpython-39.pyc
	modified:  

In [16]:
# %%
# ==============================================================================
# CELL 1: Environment and Dependencies Setup
# ==============================================================================
import os
import sys

# --- 1. Install System Libraries ---
print("--- Installing system library libgl1-mesa-glx ---")
!sudo apt-get update -qq && sudo apt-get install -y -qq libgl1-mesa-glx

# --- 2. Install Python Packages ---
print("\n--- Installing Python packages ---")
!pip install -q --upgrade pip
!pip install -q torch transformers datasets nibabel monai sentencepiece \
                einops vector-quantize-pytorch ema-pytorch beartype \
                opencv-python-headless protobuf huggingface_hub bitsandbytes pandas

# --- 3. Clone CT-CLIP Repository ---
print("\n--- Cloning GitHub repositories ---")
if not os.path.isdir("CT-CLIP"):
    !git clone https://github.com/ibrahimethemhamamci/CT-CLIP.git -q
    print("Cloned CT-CLIP.")
else:
    print("CT-CLIP repository already exists.")

# --- 4. Install CT-CLIP Subpackages ---
print("\n--- Installing CT-CLIP subpackages in editable mode ---")
!pip install -q -e CT-CLIP/transformer_maskgit
!pip install -q -e CT-CLIP/CT_CLIP

print("\n" + "="*50)
print(" ✅ Cell 1 complete.")
print(" 🔴 IMPORTANT: Please restart the kernel now before running Cell 2.")
print("="*50)

--- Installing system library libgl1-mesa-glx ---
sudo: effective uid is not 0, is /usr/bin/sudo on a file system with the 'nosuid' option set or an NFS file system without root privileges?

--- Installing Python packages ---

--- Cloning GitHub repositories ---
CT-CLIP repository already exists.

--- Installing CT-CLIP subpackages in editable mode ---
  DEPRECATION: Legacy editable install of transformer-maskgit==0.0.0 from file:///data/ct-clip-app/CT-CLIP/transformer_maskgit (setup.py develop) is deprecated. pip 25.3 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457
ERROR: pip's dependency resolver does not currently take into account all the packages that are 

In [5]:
# ==============================================================================
# CELL 2: Data Preparation, Feature Extraction (204 Samples), and Saving
# ==============================================================================
import os
import sys
import torch
import pandas as pd
import numpy as np
import monai.transforms as T
from huggingface_hub import hf_hub_download
from transformers import AutoTokenizer, AutoModel

# --- 0. Secure token access (no hardcoded tokens) ---
HF_TOKEN = os.environ.get("HF_Token")
if not HF_TOKEN:
    raise RuntimeError("HF_TOKEN missing from Secrets. Set it in Space Settings → Secrets.")

# --- 1. Add cloned repos to Python path ---
sys.path.insert(0, os.path.abspath("CT-CLIP/transformer_maskgit"))
sys.path.insert(0, os.path.abspath("CT-CLIP"))
from transformer_maskgit.MaskGITTransformer import CTViT
from ct_clip.ct_clip import CTCLIP

# --- 2. Configuration ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"--- Using device: {DEVICE} ---")

# --- 3. Load Pre-trained CT-CLIP Model ---
print("\n--- Loading CT-CLIP Model ---")
text_encoder_name = "microsoft/BiomedVLP-CXR-BERT-specialized"
text_tokenizer_clip = AutoTokenizer.from_pretrained(text_encoder_name, trust_remote_code=True)
text_encoder = AutoModel.from_pretrained(text_encoder_name, trust_remote_code=True)
image_encoder = CTViT(dim=512, codebook_size=8192, image_size=480, patch_size=20,
                     temporal_patch_size=10, spatial_depth=4, temporal_depth=4,
                     dim_head=32, heads=8)
vision_model = CTCLIP(image_encoder=image_encoder, text_encoder=text_encoder,
                      dim_image=294912, dim_text=768, dim_latent=512).to(DEVICE)

print("Downloading CT-CLIP v2 weights...")
ckpt_path = hf_hub_download(
    repo_id="ibrahimhamamci/CT-RATE",
    repo_type="dataset",
    filename="models/CT-CLIP-Related/CT-CLIP_v2.pt",
    token=HF_TOKEN,
)
state = torch.load(ckpt_path, map_location="cpu")
vision_model.load_state_dict(state.get("state_dict", state), strict=False)
vision_model.eval()
for param in vision_model.parameters():
    param.requires_grad = False
print(" ✅ CT-CLIP model loaded and frozen.")

# --- 4. Prepare Labels and Data ---
print("\n--- Preparing labels ---")
labels_path = hf_hub_download(
    repo_id="ibrahimhamamci/CT-RATE",
    repo_type="dataset",
    filename="dataset/multi_abnormality_labels/train_predicted_labels.csv",
    token=HF_TOKEN,
)
df_labels = pd.read_csv(labels_path)
CLASS_NAMES = df_labels.columns[1:].tolist()
print(f"Found {len(CLASS_NAMES)} classes.")

# --- 5. Sample filenames (balanced 204) ---
sample_filenames = [
    "train_1001_a_1.nii.gz", "train_10047_b_2.nii.gz", "train_10080_a_1.nii.gz", "train_10098_a_1.nii.gz",
    "train_10128_b_2.nii.gz", "train_10168_a_1.nii.gz", "train_1016_a_1.nii.gz", "train_10172_a_1.nii.gz",
    "train_10203_a_2.nii.gz", "train_10208_a_2.nii.gz", "train_10211_a_2.nii.gz", "train_10255_a_1.nii.gz",
    "train_10432_d_2.nii.gz", "train_10516_a_1.nii.gz", "train_10633_a_2.nii.gz", "train_10641_h_1.nii.gz",
    "train_1069_a_2.nii.gz", "train_10704_a_1.nii.gz", "train_10751_a_2.nii.gz", "train_10798_a_2.nii.gz",
    "train_10805_a_1.nii.gz", "train_10822_a_2.nii.gz", "train_10858_a_2.nii.gz", "train_10872_a_1.nii.gz",
    "train_10919_a_2.nii.gz", "train_10958_a_1.nii.gz", "train_10969_b_1.nii.gz", "train_11032_a_2.nii.gz",
    "train_11103_a_1.nii.gz", "train_11111_d_1.nii.gz", "train_11124_a_1.nii.gz", "train_11175_a_3.nii.gz",
    "train_11213_a_2.nii.gz", "train_11263_a_2.nii.gz", "train_11289_a_2.nii.gz", "train_11327_a_2.nii.gz",
    "train_11331_a_1.nii.gz", "train_11334_a_2.nii.gz", "train_1138_a_2.nii.gz", "train_11418_b_2.nii.gz",
    "train_11595_a_2.nii.gz", "train_11615_a_1.nii.gz", "train_11617_a_1.nii.gz", "train_1163_a_1.nii.gz",
    "train_11649_a_1.nii.gz", "train_11687_a_1.nii.gz", "train_11687_a_2.nii.gz", "train_11779_a_1.nii.gz",
    "train_11791_a_1.nii.gz", "train_11837_a_1.nii.gz", "train_11872_a_1.nii.gz", "train_11948_b_2.nii.gz",
    "train_12005_b_2.nii.gz", "train_12012_a_2.nii.gz", "train_1207_a_1.nii.gz", "train_12094_a_2.nii.gz",
    "train_12125_a_1.nii.gz", "train_12160_a_2.nii.gz", "train_12163_a_1.nii.gz", "train_12180_a_1.nii.gz",
    "train_12195_a_2.nii.gz", "train_12265_a_1.nii.gz", "train_12283_a_2.nii.gz", "train_12357_a_2.nii.gz",
    "train_12427_a_2.nii.gz", "train_12479_b_2.nii.gz", "train_12503_b_2.nii.gz", "train_12520_b_2.nii.gz",
    "train_12522_a_2.nii.gz", "train_12537_c_1.nii.gz", "train_12542_a_1.nii.gz", "train_12572_a_1.nii.gz",
    "train_12637_a_1.nii.gz", "train_12641_a_1.nii.gz", "train_12684_a_1.nii.gz", "train_12755_a_1.nii.gz",
    "train_12794_c_2.nii.gz", "train_12850_a_1.nii.gz", "train_12863_b_1.nii.gz", "train_12875_a_1.nii.gz",
    "train_12877_b_1.nii.gz", "train_12885_a_1.nii.gz", "train_12894_a_1.nii.gz", "train_12935_a_2.nii.gz",
    "train_12952_a_1.nii.gz", "train_12971_a_1.nii.gz", "train_129_a_1.nii.gz", "train_13016_a_1.nii.gz",
    "train_13176_b_1.nii.gz", "train_1320_a_2.nii.gz", "train_13216_a_1.nii.gz", "train_13224_a_2.nii.gz",
    "train_13251_c_1.nii.gz", "train_13252_a_1.nii.gz", "train_13355_a_2.nii.gz", "train_13355_b_1.nii.gz",
    "train_13368_b_2.nii.gz", "train_13393_a_1.nii.gz", "train_13420_a_1.nii.gz", "train_13437_d_1.nii.gz",
    "train_13438_a_2.nii.gz", "train_13448_a_2.nii.gz", "train_13473_a_2.nii.gz", "train_1348_b_2.nii.gz",
    "train_13592_a_2.nii.gz", "train_1378_c_1.nii.gz", "train_13829_a_2.nii.gz", "train_13844_a_2.nii.gz",
    "train_13849_a_1.nii.gz", "train_13854_a_1.nii.gz", "train_13962_a_2.nii.gz", "train_14046_a_2.nii.gz",
    "train_14058_b_2.nii.gz", "train_14098_a_1.nii.gz", "train_14122_a_1.nii.gz", "train_14166_d_1.nii.gz",
    "train_14268_a_1.nii.gz", "train_14270_a_1.nii.gz", "train_14370_a_1.nii.gz", "train_14392_a_1.nii.gz",
    "train_14442_h_2.nii.gz", "train_1448_c_1.nii.gz", "train_14552_a_1.nii.gz", "train_1455_a_1.nii.gz",
    "train_14600_a_1.nii.gz", "train_14643_a_1.nii.gz", "train_14684_a_1.nii.gz", "train_14688_b_1.nii.gz",
    "train_14690_a_2.nii.gz", "train_14701_b_2.nii.gz", "train_14709_a_1.nii.gz", "train_14740_b_2.nii.gz",
    "train_14764_e_2.nii.gz", "train_14787_a_2.nii.gz", "train_14829_a_1.nii.gz", "train_14835_a_2.nii.gz",
    "train_14909_a_2.nii.gz", "train_14920_a_2.nii.gz", "train_14940_a_2.nii.gz", "train_14952_a_2.nii.gz",
    "train_15014_a_2.nii.gz", "train_15020_a_1.nii.gz", "train_15021_a_2.nii.gz", "train_1504_a_1.nii.gz",
    "train_15057_a_1.nii.gz", "train_15062_a_1.nii.gz", "train_15076_b_1.nii.gz", "train_15076_b_2.nii.gz",
    "train_1516_b_2.nii.gz", "train_15200_a_1.nii.gz", "train_15225_a_1.nii.gz", "train_15302_a_2.nii.gz",
    "train_15338_a_2.nii.gz", "train_15357_a_1.nii.gz", "train_15445_a_1.nii.gz", "train_15485_a_1.nii.gz",
    "train_15508_a_1.nii.gz", "train_15639_b_2.nii.gz", "train_15654_a_1.nii.gz", "train_15672_a_2.nii.gz",
    "train_15673_a_2.nii.gz", "train_1569_a_1.nii.gz", "train_15748_a_2.nii.gz", "train_15767_a_1.nii.gz",
    "train_15858_a_1.nii.gz", "train_15867_a_2.nii.gz", "train_15909_a_2.nii.gz", "train_15911_b_1.nii.gz",
    "train_15911_b_2.nii.gz", "train_15918_a_1.nii.gz", "train_15956_b_1.nii.gz", "train_15957_a_2.nii.gz",
    "train_16006_a_1.nii.gz", "train_1600_c_2.nii.gz", "train_16049_a_1.nii.gz", "train_16173_a_1.nii.gz",
    "train_16173_a_2.nii.gz", "train_16176_b_1.nii.gz", "train_16215_a_1.nii.gz", "train_16223_a_2.nii.gz",
    "train_16276_a_2.nii.gz", "train_16294_b_2.nii.gz", "train_1629_a_1.nii.gz", "train_16307_a_2.nii.gz",
    "train_1630_a_1.nii.gz", "train_16379_a_1.nii.gz", "train_16447_a_1.nii.gz", "train_16485_a_2.nii.gz",
    "train_16503_a_2.nii.gz", "train_16512_a_1.nii.gz", "train_16554_a_1.nii.gz", "train_16562_a_1.nii.gz",
    "train_16566_b_1.nii.gz", "train_16602_a_1.nii.gz", "train_16621_a_2.nii.gz", "train_16627_a_2.nii.gz",
    "train_16641_b_2.nii.gz", "train_16649_a_2.nii.gz", "train_16664_a_1.nii.gz", "train_16724_a_2.nii.gz",
    "train_16775_a_1.nii.gz", "train_16779_a_2.nii.gz", "train_16815_b_1.nii.gz", "train_16819_a_1.nii.gz",
    "train_16865_a_1.nii.gz", "train_16886_b_2.nii.gz", "train_16887_b_2.nii.gz", "train_16887_d_1.nii.gz",
    "train_16893_a_2.nii.gz", "train_16928_a_2.nii.gz", "train_16961_a_1.nii.gz", "train_17013_a_2.nii.gz",
    "train_17038_b_1.nii.gz", "train_1708_c_1.nii.gz", "train_17155_a_2.nii.gz", "train_171_b_2.nii.gz",
    "train_17207_b_2.nii.gz", "train_17237_a_2.nii.gz", "train_17346_b_1.nii.gz", "train_17359_a_2.nii.gz",
    "train_17379_a_1.nii.gz", "train_1742_c_1.nii.gz", "train_17459_a_1.nii.gz", "train_1746_a_1.nii.gz",
    "train_17507_a_1.nii.gz", "train_17508_a_1.nii.gz", "train_17531_a_5.nii.gz", "train_17534_a_1.nii.gz",
    "train_1753_e_2.nii.gz", "train_17553_a_1.nii.gz", "train_17610_a_1.nii.gz", "train_17634_b_1.nii.gz",
    "train_17732_b_2.nii.gz", "train_17737_a_2.nii.gz", "train_17750_a_2.nii.gz", "train_17786_a_2.nii.gz",
    "train_17828_a_2.nii.gz", "train_17835_a_2.nii.gz", "train_17877_a_1.nii.gz", "train_18010_a_2.nii.gz",
    "train_18016_a_1.nii.gz", "train_18059_a_2.nii.gz", "train_18062_a_2.nii.gz", "train_18084_a_2.nii.gz",
    "train_18119_a_1.nii.gz", "train_18130_b_2.nii.gz", "train_18136_a_2.nii.gz", "train_18207_b_2.nii.gz",
    "train_18250_a_2.nii.gz", "train_18267_b_2.nii.gz", "train_18312_d_2.nii.gz", "train_18452_a_1.nii.gz",
    "train_1845_a_5.nii.gz", "train_1845_b_1.nii.gz", "train_1845_c_2.nii.gz", "train_18465_b_2.nii.gz",
    "train_18469_a_2.nii.gz", "train_18470_a_1.nii.gz", "train_1847_a_1.nii.gz", "train_18625_a_1.nii.gz",
    "train_18663_a_2.nii.gz", "train_18668_a_2.nii.gz", "train_18678_a_2.nii.gz", "train_18744_a_1.nii.gz",
    "train_18838_b_2.nii.gz", "train_18877_a_1.nii.gz", "train_18983_b_1.nii.gz", "train_19015_b_2.nii.gz",
    "train_19041_a_2.nii.gz", "train_19061_a_1.nii.gz", "train_19070_a_1.nii.gz", "train_19088_d_2.nii.gz",
    "train_19105_b_2.nii.gz", "train_19123_a_2.nii.gz", "train_19214_a_2.nii.gz", "train_19220_a_2.nii.gz",
    "train_19230_a_1.nii.gz", "train_19244_c_1.nii.gz", "train_19263_a_2.nii.gz", "train_19269_a_2.nii.gz",
    "train_19283_a_2.nii.gz", "train_19350_a_2.nii.gz", "train_19379_a_2.nii.gz", "train_19486_a_2.nii.gz",
    "train_19512_a_2.nii.gz", "train_19551_a_1.nii.gz", "train_19578_a_2.nii.gz", "train_19640_a_1.nii.gz",
    "train_19695_a_2.nii.gz", "train_19695_c_1.nii.gz", "train_19695_d_1.nii.gz", "train_1969_a_1.nii.gz",
    "train_19819_a_2.nii.gz", "train_19844_a_2.nii.gz", "train_19845_a_2.nii.gz", "train_19874_a_2.nii.gz",
    "train_19884_a_1.nii.gz", "train_19885_e_2.nii.gz", "train_19893_b_2.nii.gz", "train_1993_a_1.nii.gz",
    "train_2002_a_1.nii.gz", "train_2107_a_2.nii.gz", "train_220_a_1.nii.gz", "train_2299_b_2.nii.gz",
    "train_2410_d_2.nii.gz", "train_2423_a_2.nii.gz", "train_2650_a_2.nii.gz", "train_2755_a_1.nii.gz",
    "train_2776_a_1.nii.gz", "train_2790_b_2.nii.gz", "train_2838_a_1.nii.gz", "train_2855_a_3.nii.gz",
    "train_288_a_1.nii.gz", "train_2892_a_2.nii.gz", "train_2916_a_1.nii.gz", "train_3067_a_2.nii.gz",
    "train_3089_a_2.nii.gz", "train_3113_a_2.nii.gz", "train_3114_a_3.nii.gz", "train_3138_a_2.nii.gz",
]

# --- 6. Preprocessing and Feature Extraction ---
def load_and_preprocess_ct(path: str):
    transform = T.Compose([
        T.LoadImaged(keys=["image"]), T.EnsureChannelFirstd(keys=["image"]),
        T.ScaleIntensityRanged(keys=["image"], a_min=-1000, a_max=1000,
                               b_min=-1.0, b_max=1.0, clip=True),
        T.Resized(keys=["image"], spatial_size=(40, 480, 480),
                  mode="trilinear", align_corners=False),
        T.ToTensord(keys=["image"]),
    ])
    data = transform({"image": path})
    return data["image"].unsqueeze(0).to(DEVICE)

def reconstruct_path_from_filename(filename: str) -> str:
    parts = filename.replace('.nii.gz', '').split('_')
    train_id = parts[1]
    scan_id = parts[2]
    return f"dataset/train/train_{train_id}/train_{train_id}_{scan_id}/{filename}"

print("\n--- Extracting features for our mini-dataset ---")
training_data = []
for filename in sample_filenames:
    try:
        full_path = reconstruct_path_from_filename(filename)
        print(f"Processing {full_path}...")
        local_path = hf_hub_download(
            repo_id="ibrahimhamamci/CT-RATE",
            repo_type="dataset",
            filename=full_path,
            token=HF_TOKEN,
        )
        ct_tensor = load_and_preprocess_ct(local_path)

        with torch.no_grad():
            dummy_text_input = text_tokenizer_clip([""], return_tensors="pt", padding=True).to(DEVICE)
            _, image_embedding, _ = vision_model(dummy_text_input, ct_tensor, DEVICE, return_latents=True)

        labels_row = df_labels[df_labels['VolumeName'] == filename]
        if not labels_row.empty:
            label_vector = torch.tensor(labels_row.iloc[0, 1:].values.astype(np.float32)).to(DEVICE)
            training_data.append((image_embedding, label_vector))
    except Exception as e:
        print(f"Could not process file {filename}. Error: {e}. Skipping.")

print(f"\n ✅ Cell 2 complete. Created a mini-dataset with {len(training_data)} samples.")

# === 7. Save extracted features and labels for later reuse ===
os.makedirs("saved_artifacts", exist_ok=True)
image_embeddings = torch.cat([item[0] for item in training_data], dim=0)  # [N, 512]
label_vectors = torch.stack([item[1] for item in training_data], dim=0)   # [N, num_classes]

torch.save({
    "image_embeddings": image_embeddings,
    "label_vectors": label_vectors,
    "class_names": CLASS_NAMES,
}, "saved_artifacts/features_labels.pt")

# Save sample filename manifest
with open("saved_artifacts/sample_filenames.txt", "w") as f:
    for name in sample_filenames:
        full_path = reconstruct_path_from_filename(name)
        f.write(f"{name}\t{full_path}\n")

# Save a human-readable summary
summary_rows = []
for i, (emb, label) in enumerate(training_data):
    summary_rows.append({
        "sample_index": i,
        "volume_name": sample_filenames[i],
        "embedding_norm": emb.norm().item(),
        **{f"class_{j}": label[j].item() for j in range(label.shape[0])}
    })
df_summary = pd.DataFrame(summary_rows)
df_summary.to_csv("saved_artifacts/training_summary.csv", index=False)

print("✅ Saved features+labels, sample list, and summary to `saved_artifacts/`")


--- Using device: cuda ---

--- Loading CT-CLIP Model ---


tokenizer_config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

configuration_cxrbert.py:   0%|          | 0.00/889 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/235k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

dataset/train/train_10168/train_10168_a/(…):   0%|          | 0.00/130M [00:00<?, ?B/s]

test all pooling
Processing dataset/train/train_1016/train_1016_a/train_1016_a_1.nii.gz...


dataset/train/train_1016/train_1016_a/tr(…):   0%|          | 0.00/475M [00:00<?, ?B/s]

Cancellation requested; stopping current tasks.


KeyboardInterrupt: 